In [1]:
import numpy as np
import pandas as pd 
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd
import osmnx as ox
import itertools as it
%matplotlib inline

In [134]:
flow = 10

In [135]:
G = nx.MultiDiGraph()
travel_time_awal = [10,20,25]
key = G.add_edge(1,2, capacity = 2)
key2 = G.add_edge(1,2, capacity = 4)
key3 = G.add_edge(1,2, capacity = 3)


0

In [136]:
df = nx.to_pandas_edgelist(G)
df['travel_time_awal'] = travel_time_awal
df['travel_time'] = travel_time_awal
df['flow'] = [0,0,0]
df
# edge_attr = df_attributes_only.set_index('id').to_dict('index')

,source,target,capacity,travel_time_awal,travel_time,flow
0,1,2,2,10,10,0
1,1,2,4,20,20,0
2,1,2,3,25,25,0


In [137]:
# nx.set_edge_attributes(G, values = df['travel_time'].values, name="travel_time")
G = nx.from_pandas_edgelist(df, source='source', target='target', edge_attr=True, create_using=nx.MultiDiGraph())
G[1][2]

AtlasView({0: {'capacity': 2, 'travel_time_awal': 10, 'travel_time': 10, 'flow': 0}, 1: {'capacity': 4, 'travel_time_awal': 20, 'travel_time': 20, 'flow': 0}, 2: {'capacity': 3, 'travel_time_awal': 25, 'travel_time': 25, 'flow': 0}})

In [198]:
routes = nx.shortest_path(G, source=1, target=2, weight='travel_time')
routes

[1, 2]

In [139]:
minimum = G[1][2][0]['travel_time']
keys = 0
for index, i in enumerate(G[1][2]):
    if G[1][2][i]['travel_time'] < minimum:
        minimum = G[1][2][i]
        keys = index

keys

0

In [ ]:
def routes_jalan(G, routes):
    list_jalan = []
    for i in range(len(routes)-1):
        minimum = G[routes[i]][routes[i+1]][0]['travel_time']
        n = 0
        for node1, node2, data in G.edges(data=True):
            if data['travel_time'] < minimum:
                minimum = data['travel_time']
                keys = n
            n += 1
        list_jalan.append(keys)
    
    return list_jalan

In [199]:
rute_jalan_key = routes_jalan(G, routes)
rute_jalan_key

947.5
<class 'float'>
<class 'float'>
1 2 0 947.5
<class 'float'>
1 2 1 20.0


TypeError: '<=' not supported between instances of 'float' and 'dict'

## Updating Flow

In [230]:
nx.set_edge_attributes(G, values = 0, name='auxflow')

In [234]:
def updating_mainflow(G, routes):

    rute_jalan_key = routes_jalan(G, routes)

    for i in range(len(rute_jalan_key)):
        attrs = {(routes[i], routes[i+1], rute_jalan_key[i]): {"flow": flow}}
        nx.set_edge_attributes(G, attrs)

    return G  

In [235]:
def updating_auxflow(G, routes):

    rute_jalan_key = routes_jalan(G, routes)

    for i in range(len(rute_jalan_key)):
        attrs = {(routes[i], routes[i+1], rute_jalan_key[i]): {"auxflow": flow}}
        nx.set_edge_attributes(G, attrs)

    return G  

In [145]:
nx.to_pandas_edgelist(G)

,source,target,flow,capacity,travel_time,travel_time_awal
0,1,2,0,2,10,10
1,1,2,0,4,20,20
2,1,2,0,3,25,25


In [146]:
Gupdate = updating_flow(G, routes)
df = nx.to_pandas_edgelist(Gupdate)
df

,source,target,flow,capacity,travel_time,travel_time_awal
0,1,2,10,2,10,10
1,1,2,0,4,20,20
2,1,2,0,3,25,25


In [147]:
#This is a link performance function derived from Indonesia's Highway Manual (MKJI 1997).
def lpr_idn(row):
    if np.isnan(row['flow']):
        return row['travel_time']
    else:
        try:
            return row['travel_time_awal']*(1+0.15*(row['flow']/row['capacity'])**4)
        except ValueError:
            return row['travel_time']

In [148]:
list_edge = []
for i in G.edges():
    list_edge.append(i)

for index, i in enumerate(list_edge):
    attrib = {(i[0], i[1], index): {"travel_time": lpr_idn(G[i[0]][i[1]][index])}}
    nx.set_edge_attributes(G, attrib)

nx.to_pandas_edgelist(G)

,source,target,flow,capacity,travel_time,travel_time_awal
0,1,2,10,2,947.5,10
1,1,2,0,4,20.0,20
2,1,2,0,3,25.0,25


In [192]:
routes = nx.shortest_path(G, source=1, target=2, weight='travel_time')
routes

[1, 2]

In [236]:
rute_jalan_key = routes_jalan(G, routes)
rute_jalan_key

[1]

In [237]:
Gupdate = updating_auxflow(G, routes)
nx.to_pandas_edgelist(Gupdate)

,source,target,auxflow,travel_time_awal,capacity,travel_time,flow
0,1,2,0,10,2,947.5,10
1,1,2,10,20,4,20.0,0
2,1,2,0,25,3,25.0,0


## Bisection Method

In [240]:
def line_search(row, a):
    return (row['auxflow']-row['flow'])*row['travel_time']*(row['flow']+a*(row['auxflow']-row['flow']))

In [262]:
def bisection(G, xl=0, xr=1, delta=0.01):
    n = 0
    df = nx.to_pandas_edgelist(G)
    condition = True
    while condition:
        n += 1
        x = (xl+xr)/2
        if df.apply(lambda row: line_search(row, x), axis = 1).sum() <= 0:
            xl = x
        else:
            xr = x
        condition = abs(xr-xl) > 2*delta
        print('|xl-xr| =', abs(xr-xl))
    print(f"number of iteration in bisection method is {n}")
    return (xr+xl)/2

In [271]:
nx.to_pandas_edgelist(G)

,source,target,auxflow,travel_time_awal,capacity,travel_time,flow
0,1,2,0,10,2,947.5,10
1,1,2,10,20,4,20.0,0
2,1,2,0,25,3,25.0,0


In [280]:
z = 

0   -38279.0
1     1192.0
2        0.0
dtype: float64

-47375.0